# Preprocessing: procedures_icd Table

In [1]:
import os
os.chdir('../')
from utils.hosp_preprocess_util import *
import pickle

### Getting the Procedures table into long format with timedelta

In [2]:
# Use custom function to get timedelta according to the procedure's chartdate
proc = timestamp_cohort_data("./mimic-iv-1.0/hosp/procedures_icd.csv.gz", './data/cohort.gzip', 'chartdate', 'base_anchor_year', dtypes=None, usecols=None)
proc.head()

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,admit_year,label,base_anchor_year,max_year_group,min_year_group,anchor_year,timedelta_days,timedelta_years
516275,10000032,25742920,1,2180-08-06,5491,9,2180,0,2174,2016,2014,2180,2409,6.0
516273,10000032,22595853,1,2180-05-07,5491,9,2180,0,2174,2016,2014,2180,2318,6.0
516274,10000032,22841357,1,2180-06-27,5491,9,2180,0,2174,2016,2014,2180,2369,6.0
473014,10000280,25852320,1,2151-03-18,8938,9,2151,0,2151,2010,2008,2151,76,0.0
248724,10000560,28979390,1,2189-10-16,5551,9,2189,0,2189,2010,2008,2189,288,0.0


In [3]:
# Print unique counts and value_counts
print("# Unique ICD9 Procedures:  ", proc.loc[proc.icd_version == 9].icd_code.dropna().nunique())
print("# Unique ICD10 Procedures: ",proc.loc[proc.icd_version == 10].icd_code.dropna().nunique())

print("\nValue counts of each ICD version:\n", proc.icd_version.value_counts())

# Unique ICD9 Procedures:   2549
# Unique ICD10 Procedures:  4932

Value counts of each ICD version:
 9     466147
10     44906
Name: icd_version, dtype: int64


### Saving the long-format dataset

In [4]:
# Save two versions of procedures dataset; one with all procedure codes (ICD9 & 10), and then one with only ICD10 codes
proc[['subject_id', 'hadm_id', 'icd_code', 'timedelta_days']].dropna().to_pickle("./data/long_format/proc/long_proc.gzip", compression='gzip')
proc.loc[proc.icd_version == 10][['subject_id', 'hadm_id', 'icd_code', 'timedelta_days']].dropna().to_pickle("./data/long_format/proc/long_proc_icd10.gzip", compression='gzip')

### One Hot Encoding and Pivoting

Old code for pivoting long_format data into the form:
```
                                    || feature 1 || ... || feature n ||
|| subject_id || label || timedelta ||
```

In [2]:
# proc = preproc_icd_module("./mimic-iv-1.0/hosp/procedures_icd.csv.gz", './data/adm_cohort.gzip')

In [5]:
# proc

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,label,timedelta
0,10287061,27485182,1,2124-08-22,7569,9,0,0.0
1,13135573,22286210,3,2187-05-30,7309,9,0,0.0
2,13135573,22286210,1,2187-05-30,7569,9,0,0.0
3,13135573,22286210,2,2187-05-30,734,9,0,0.0
4,12646116,24154012,2,2115-01-13,734,9,0,0.0
...,...,...,...,...,...,...,...,...
585387,16889089,25347452,3,2170-10-19,02HV33Z,10,1,9.0
585388,16889089,25347452,4,2170-10-24,3E0G76Z,10,1,9.0
585389,16889089,25347452,5,2170-10-25,0BH17EZ,10,1,9.0
585390,16889089,25347452,6,2170-10-25,5A1945Z,10,1,9.0


In [6]:
# pivot_proc = pivot_cohort(proc[['subject_id', 'label','timedelta', 'icd_code']], prefix='proc_', target_col='icd_code', ohe=True, max_features=100)
# pivot_proc_icd10 = pivot_cohort(proc.loc[proc.icd_version == 10][['subject_id', 'label','timedelta', 'icd_code']], prefix='proc_', target_col='icd_code', ohe=True, max_features=100)

In [7]:
# # Save output
# pivot_proc.fillna(0).astype(int).to_pickle('./data/proc/pivot_proc.gzip', compression='gzip')
# pivot_proc_icd10.fillna(0).astype(int).to_pickle('./data/proc/pivot_proc_icd10.gzip', compression='gzip')